# Implementation of Box Parameterization Algorithm.

In [2]:
from importlib.util import find_spec
if find_spec("model") is None:
    import sys
    sys.path.append('..')

In [4]:
import math
from typing import Tuple
import torch

## Parameterization for BBox Regression.
paper: [Rich feature hierarchies for accurate object detection and semantic segmentation](https://arxiv.org/abs/1311.2524)

In [5]:
# Value for clamping large dw and dh predictions. The heuristic is that we clamp
# such that dw and dh are no larger than what would transform a 16px box into a
# 1000px box (based on a small anchor, 16px, and a typical image size, 1000px).
_DEFAULT_SCALE_CLAMP = math.log(1000.0 / 16)

In [7]:
class Box2BoxTransform:
    """
    The box-to-box transform defined in R-CNN. The transformation is parameterized
    by 4 deltas: (dx, dy, dw, dh). The transformation scales the box's width and height
    by exp(dw), exp(dh) and shifts a box's center by the offset (dx * width, dy * height).
    """
    
    def __init__(
        self, weights: Tuple[float, float, float, float], scale_clamp: float = _DEFAULT_SCALE_CLAMP
    ):
        """
        Args:
            weights: Scaling factors that are applied to the (dx, dy, dw, dh) deltas.
                In "Fast R-CNN" these were originally set such that the deltas have unit var.
                Here they are treated as hyperparameters of the system.
            scale_clamp: When predicting deltas the predicted box scaling factors (dw, dh) are
                clamped such that they are <= scale_clamp.
        """
        self.weights = weights
        self.scale_clamp = scale_clamp
        
    def get_deltas(self, src_boxes, target_boxes):
        """
        Get deltas as defined in R-CNN paper between `src_boxes` and `target_boxes`.
        
        Calculate box regression transformation deltas (dx, dy, dw, dh) to be used
        to transform the `src_boxes` into the `target_boxes`.
        
        That is, the relation ``target_boxes == self.apply_deltas(deltas, src_boxes)``
        is true (unless any delta is too large and is clamped).

        Args:
            src_boxes (Tensor): source boxes, e.g., object proposals in boundary coords.
            target_boxes (Tensor): target of the transformation, e.g., ground-truth
                boxes in boundary coords.
        """
        assert isinstance(src_boxes, torch.Tensor), type(src_boxes)
        assert isinstance(target_boxes, torch.Tensor), type(target_boxes)
        
        src_widths = src_boxes[:, 2] - src_boxes[:, 0]
        src_heights = src_boxes[: 3] - src_boxes[:, 1]
        src_ctr_x = src_boxes[:, 0] + 0.5 * src_widths
        src_ctr_y = src_boxes[: 1] + 0.5 * src_heights
        
        target_widths = target_boxes[:, 2] - target_boxes[:, 0]
        target_heights = target_boxes[: 3] - target_boxes[:, 1]
        target_ctr_x = target_boxes[:, 0] + 0.5 * target_widths
        target_ctr_y = target_boxes[: 1] + 0.5 * target_heights
        
        wx, wy, ww, wh = self.weights
        dx = wx * (target_ctr_x - src_ctr_x) / src_widths
        dy = wy * (target_ctr_y - src_ctr_y) / src_heights
        dw = ww * torch.log(target_widths / src_widths)
        dh = wh * torch.log(target_heights / src_heights)
        
        deltas = torch.stack((dx, dy, dw, dh), dim=1)
        
        assert (src_widths > 0).all().item(), "Input boxes to Box2BoxTransform are not valid!" 
        return deltas
    
    def apply_deltas(self, deltas, boxes):
        """
        Apply transformation `deltas` to `boxes`.
        
        Args:
            deltas (Tensor): transformation deltas of shape (N, k*4), where k >=1.
                deltas[i] represents k potentially different class-specific box 
                transformations for the single box boxes[i].
            boxes (Tensor): boxes to transform, of shape (N, 4)
        """
        
        deltas = deltas.float()  # ensure fp32 for decoding precision
        boxes = boxes.to(deltas.dtype)
        
        widths = boxes[:, 2] - boxes[:, 0]
        heights = boxes[:, 3] - boxes[:, 1]
        ctr_x = boxes[:, 0] + 0.5 * widths
        ctr_y = boxes[:, 1] + 0.5 * heights
        
        wx, wy, ww, wh = self.weights
        dx = deltas[:, 0::4] / wx
        dy = deltas[:, 1::4] / wy
        dw = deltasl[:, 2::4] / ww        
        dh = deltas[:, 3::4] / wh
        
        # Prevent sending too large values into torch.exp()
        dw = torch.clamp(dw, max=self.scale_clamp) 
        dh = torch.clamp(dh, max=self.scale_clamp) 
        
        pred_ctr_x = dx * widths[:, None] + ctr_x[:, None]
        pred_ctr_y = dy * heights[:, None] + ctr_y[:, None]
        pred_w = torch.exp(dw) * widths[:, None]
        pred_h = torch.exp(dh) * heights[:, None]
        
        pred_boxes = torch.zeros_like(deltas)
        pred_boxes[:, 0::4] = pred_ctr_x - 0.5 * pred_w
        pred_boxes[:, 1::4] = pred_ctr_y - 0.5 * pred_h
        pred_boxes[:, 2::4] = pred_ctr_x + 0.5 * pred_w
        pred_boxes[:, 3::4] = pred_ctr_y + 0.5 * pred_h

In [8]:
box_trfm = Box2BoxTransform([1., 1., 1., 1.])